# Leaf Disease Classification CNN Model

---

This is a ML model created by Keshav Ghai (An aspiring AI/ML dev).

This is a **Convolutional Neural Network (CNN)** which classifies plant leaf diseases across multiple crop types. The model learns hierarchical visual features from leaf images to identify disease patterns, discoloration, spots, and other disease-specific indicators. Unlike previous models that worked with single-class classification, this model performs **multi-class classification** with a plant + disease mapping system. The training script **"trainer.py"** takes image data, preprocesses it, trains a CNN with 3 convolutional blocks, and generates visualizations.

## What makes this different?

Traditional disease identification requires domain expertise. This model automates the process by learning disease-specific visual patterns. It combines:
- **Hierarchical feature extraction** through 3 CNN blocks (32→64→128 filters)
- **Plant + Disease mapping** for interpretable, hierarchical predictions
- **Multi-crop support** - identifies both the plant type and specific disease

## Imports:-
---

In [ ]:
import os
import json
import re
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import cv2

## 0. Dataset Normalization (Data Preprocessing)
---

Before actually starting the model, we have to normalize the dataset. It is done through these steps:

**Why normalize?** Raw leaf images come in different sizes, orientations, and lighting conditions. Normalization standardizes them so the model can process them consistently.

### Normalization Steps:
1. **Read images** using OpenCV from the dataset folder
2. **Convert color space** from BGR (OpenCV's default) to RGB (standard format)
3. **Resize to 256×256** - ensures all images have the same dimensions
4. **Save normalized images** to a new clean directory

The `normalization.py` script handles this automatically, creating a `normalized_dataset` folder with properly formatted images ready for training.

In [ ]:
# normalization.py

import os
import cv2
import numpy as np

DATASET_DIR = "./tensorflow/leaf_disease/dataset"
OUTPUT_DIR = "./tensorflow/leaf_disease/normalized_dataset"
TARGET_SIZE = (256, 256)

def ensure_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def normalize_images():
    ensure_dir(OUTPUT_DIR)
    
    # Create Train and Val directories
    for split in ['Train', 'Val']:
        for item in os.listdir(os.path.join(DATASET_DIR, split)):
            input_path = os.path.join(DATASET_DIR, split, item)
            output_path = os.path.join(OUTPUT_DIR, split, item)
            ensure_dir(output_path)
            
            for img_name in os.listdir(input_path):
                img_path = os.path.join(input_path, img_name)
                img = cv2.imread(img_path)
                
                if img is None:
                    continue
                
                # Convert BGR → RGB
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                
                # Resize to 256×256
                img = cv2.resize(img, TARGET_SIZE)
                
                # Save normalized image
                out_path = os.path.join(output_path, img_name)
                cv2.imwrite(out_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    
    print("Normalization complete! All images resized to 256x256.")

if __name__ == "__main__":
    normalize_images()

## 1. Define Paths and Constants
---

In [ ]:
DATASET_DIR = "./tensorflow/leaf_disease/normalized_dataset"
MODEL_SAVE_PATH = "./Models/leaf_disease_model.keras"
CLASS_MAP_PATH = "./tensorflow/leaf_disease/class_indices.json"

IMG_SIZE = (256, 256)
BATCH_SIZE = 32
EPOCHS = 13

print(f"Dataset directory: {DATASET_DIR}")
print(f"Model will be saved to: {MODEL_SAVE_PATH}")

## 2. Dataset Helper Function
---

Creating a helper function to clean disease names for display (converting underscores to spaces and proper capitalization).

In [ ]:
def clean_disease_name(raw):
    """Convert raw disease name to readable format."""
    name = raw.replace("_", " ")
    name = re.sub(r"\s+", " ", name).strip()

    tokens = name.split(" ")
    rebuilt = []
    current = []

    for token in tokens:
        if not token:
            continue
        if token[0].isupper() and current:
            rebuilt.append(" ".join(current))
            current = [token]
        else:
            current.append(token)

    if current:
        rebuilt.append(" ".join(current))

    final = ", ".join(rebuilt)
    final = final.title()

    final = re.sub(
        r"\(([^)]+)\)",
        lambda m: "(" + m.group(1).replace("_", " ").title() + ")",
        final
    )

    return final

# Test the function
print(clean_disease_name("Early_Blight"))
print(clean_disease_name("Late_Blight"))

## 3. Load Training and Validation Datasets
---

> Images are loaded from directories using Keras' `image_dataset_from_directory()`. This function automatically handles:
> - Reading images from subdirectories (each subdirectory = a class)
> - Converting images to tensors
> - Batching the data
> - Integer label mode for multi-class classification

In [ ]:
# To ensure consistent class ordering between Train and Val, compute class list once
train_dir = os.path.join(DATASET_DIR, "Train")
class_list = sorted([d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))])

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(DATASET_DIR, "Train"),
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="int",
    class_names=class_list
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    os.path.join(DATASET_DIR, "Val"),
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="int",
    class_names=class_list
)

class_names = train_ds.class_names
print(f"Number of classes: {len(class_names)}")
print(f"Sample classes: {class_names[:5]}")

# Sanity check: ensure both datasets use the same class ordering
assert train_ds.class_names == val_ds.class_names, "Train/Val class ordering mismatch!"

## 4. Build Class Mapping (Plant + Disease)
---

> Class names follow the format: `Plant___Disease`
> We extract plant and disease, then save a JSON mapping for later inference.

In [ ]:
def build_class_map():
    mapping = {}

    for idx, cls in enumerate(class_names):
        # Safely partition by delimiter to avoid ValueError
        plant, sep, disease_raw = cls.partition("___")
        if sep == "":
            # Missing delimiter - log and default disease_raw to empty
            print(f"Warning: class name '{cls}' missing '___' delimiter. Skipping detailed disease parsing.")
            disease_raw = ""

        disease_clean = clean_disease_name(disease_raw) if disease_raw else ""

        mapping[cls] = {
            "index": idx,
            "plant": plant,
            "disease_raw": disease_raw,
            "disease": disease_clean
        }

    with open(CLASS_MAP_PATH, "w") as f:
        json.dump(mapping, f, indent=4)

    return mapping

class_map = build_class_map()

print(f"Sample mappings:")
for i, (key, val) in enumerate(list(class_map.items())[:3]):
    print(f"  {key}: {val}")

## 5. Prefetch for Performance
---

> Prefetch optimizes data loading by preparing batches while the model trains.

In [ ]:
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

print("Datasets prefetched for optimal performance.")

## 6. Model Architecture
---

> The model uses 3 convolutional blocks that progressively extract and combine features:
> - **Block 1 (32 filters)**: Detects low-level features (edges, textures)
> - **Block 2 (64 filters)**: Combines features, detects patterns
> - **Block 3 (128 filters)**: Learns complex disease-specific patterns

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3), name="input"),
    tf.keras.layers.Rescaling(1.0/255.0),
    
    # Block 1: 32 filters
    tf.keras.layers.Conv2D(32, (3, 3), activation="relu", padding="same"),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    # Block 2: 64 filters
    tf.keras.layers.Conv2D(64, (3, 3), activation="relu", padding="same"),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    # Block 3: 128 filters
    tf.keras.layers.Conv2D(128, (3, 3), activation="relu", padding="same"),
    tf.keras.layers.MaxPooling2D((2, 2)),
    
    # Dense layers
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(len(class_names), activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

## 7. Train the Model
---

> The model is trained for 13 epochs with a batch size of 32.

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    verbose=1
)

# Ensure model save directory exists
model_dir = os.path.dirname(MODEL_SAVE_PATH)
if model_dir:
    os.makedirs(model_dir, exist_ok=True)

model.save(MODEL_SAVE_PATH)
print(f"Model saved to {MODEL_SAVE_PATH}")

## 8. Training Visualizations
---

### a. Loss Over Epochs

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Val Loss")
plt.title("Loss Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.savefig("./tensorflow/leaf_disease/loss_graph.png")
plt.show()
plt.close()

### b. Accuracy Over Epochs

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(history.history["accuracy"], label="Train Accuracy")
plt.plot(history.history["val_accuracy"], label="Val Accuracy")
plt.title("Accuracy Over Epochs")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.savefig("./tensorflow/leaf_disease/accuracy_graph.png")
plt.show()
plt.close()

### c. Confusion Matrix

In [ ]:
# Get all validation predictions
val_pred_raw = model.predict(val_ds)
val_pred_classes = np.argmax(val_pred_raw, axis=1)

# Get true labels
val_labels = np.concatenate([y for x, y in val_ds])

# Confusion matrix
cm = confusion_matrix(val_labels, val_pred_classes)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=True)
plt.title("Confusion Matrix - Leaf Disease Classification")
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.savefig("./tensorflow/leaf_disease/confusion_matrix.png", dpi=150, bbox_inches="tight")
plt.show()
plt.close()

print("Saved: loss_graph.png, accuracy_graph.png, confusion_matrix.png")

## 9. Interactive Testing
---

> Test the model on individual leaf images with plant + disease predictions.

In [ ]:
# Create reverse mapping for inference
index_to_class = {v['index']: k for k, v in class_map.items()}

print("\nInteractive testing mode:")
print("(Run this cell and enter image paths to test the model)\n")

while True:
    img_path = input("Enter leaf image path (or 'quit'): ").strip()
    
    if img_path.lower() == "quit":
        break
    
    if not os.path.exists(img_path):
        print("File not found, try again.\n")
        continue
    
    try:
        # Load and preprocess image
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=IMG_SIZE)
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_batch = np.expand_dims(img_array, axis=0)
        
        # Make prediction
        pred = model.predict(img_batch, verbose=0)[0]
        class_id = np.argmax(pred)
        class_name = index_to_class[class_id]
        confidence = float(np.max(pred) * 100)
        
        # Extract info
        plant = class_map[class_name]['plant']
        disease = class_map[class_name]['disease']
        
        print(f"\nPrediction:")
        print(f"  Plant: {plant}")
        print(f"  Disease: {disease}")
        print(f"  Confidence: {confidence:.2f}%\n")
    
    except Exception as e:
        print(f"Error: {e}\n")